**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 8.6MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N50ff2fb1b59d49eda83cdfc1222f808e (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [10]:
from rdflib.plugins.sparql import prepareQuery

ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s)

q1 = prepareQuery('''
  SELECT ?subclass
  WHERE { 
    ?subclass rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = {"rdfs":RDFS, "ns":ns}
)

for s in g.query(q1):
  print(s)

http://somewhere#Researcher
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [11]:
from rdflib.plugins.sparql import prepareQuery

for s,p,o in g.triples((None, RDF.type, ns.Person)) :
  print(s)
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for a,b,c in g.triples((None, RDF.type, s)):
    print(a)

q2 = prepareQuery('''
  SELECT ?s
  WHERE { 
    {
      ?s rdf:type ns:Person. 
    }
    UNION {
      ?ss (rdfs:subClassOf/rdfs:subClassOf*) ns:Person.
      ?s rdf:type ?ss
    }
  }
  
  ''',
  initNs = {"rdfs":RDFS, "rdf":RDF, "ns":ns}
)

for s in g.query(q2):
  print(s)

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [12]:
from rdflib.plugins.sparql import prepareQuery

for s,p,o in g.triples((None, RDF.type, ns.Person)) :
  for s2, p2, o2 in g.triples((s, None, None)):
    print(s2, p2, o2)
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s2,p2,o2 in g.triples((None, RDF.type, s)):
    for s3, p3, o3 in g.triples((s2, None, None)):
      print(s3, p3, o3)
      
q3 = prepareQuery('''
  SELECT ?s ?p ?z
  WHERE { 
    {
      ?s rdf:type ns:Person. 
      ?s ?p ?z
    }
    UNION {
      ?ss (rdfs:subClassOf/rdfs:subClassOf*) ns:Person.
      ?s rdf:type ?ss.
      ?s ?p ?z
    }
  }
  
  ''',
  initNs = {"rdfs":RDFS, "rdf":RDF, "ns":ns}
)

for s in g.query(q3):
  print(s)

http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/